In [ ]:
#!/usr/bin/env python

import rospy,actionlib,ipywidgets as widgets

from my_assignment.msg import PlanningAction, PlanningGoal,Posvel

from nav_msgs.msg import Odometry

from my_assignment.srv import Counter

from ipywidgets import Button, ButtonStyle, Layout, GridBox, Text

from IPython.display import display

from ipywidgets import interact, interactive, fixed,interact_manual

x=0
y=0

b1=Button(description="choose goal",layout=Layout(width="auto",align="center",grid_area="b1"),
                 style=ButtonStyle(button_color="lightblue"))

b2=Button(description="cancel goal",layout=Layout(width="auto",grid_area="b2"),
                 style=ButtonStyle(button_color="moccasin"))

b3=Button(description="reached/cancelled goals",layout=Layout(width="auto",grid_area="b3"),
                 style=ButtonStyle(button_color="salmon"))

b4=Button(description="exit",layout=Layout(width="auto",grid_area="b4"),
                 style=ButtonStyle(button_color="green"))



def x_cb(wd):
    
    x=wd.value
    
def y_cb(wd):
    
    y=wd.value
    
    goals()


def position(b): 
    
    global x,y

    x_text = widgets.Text(
            description='X:',
            disabled=False
            )
    
    display(x_text)
    
    x_text.on_submit(x_cb)
    
    y_text = widgets.Text(
            description='Y:',
            disabled=False
            )
    

    display(y_text)
    
    y_text.on_submit(y_cb)


def goals():
    
    
    #client.send_goal(goal) #sending goal to the server
    
    #goal = position() #getting goal coordinates
   
    
    goal=PlanningGoal()  #initialising goal to the message PlanningGoal

    goal.target_pose.pose.position.x=x #set the x goal position

    goal.target_pose.pose.position.y=y #set the y goal position
    
    
    
    #goal=PlanningGoal()  #initialising goal to the message PlanningGoal

    #goal.target_pose.pose.position.x=2 #set the x goal position

    #goal.target_pose.pose.position.y=2 #set the y goal position
    
    client.wait_for_server()
    
    client.send_goal(goal) #sending goal to the server
    
    rospy.loginfo("Goal sent!\n")
    
   
        

def cancel(b):
    
    client.wait_for_server()
    client.cancel_goal()
    

def reached_cancelled(b):
    
    rospy.wait_for_service('counter') #synchronizing with service node
    service=rospy.ServiceProxy('counter',Counter) #request to the service node 
    counter=service("ok") # used a message "ok" to avoid any problem with empty message
    rospy.loginfo("reached goal: %s and cancelled goal: %s\n" %(counter.total_reach,counter.total_cancel))
    

    
def publish_function(od): 

    pub=rospy.Publisher('/posvel',Posvel,queue_size=10) # publisher on /posvel topic

    info=Posvel() #initialising info to the custom message Posvel

    info.x=od.pose.pose.position.x #get x position from /odom topic

    info.y=od.pose.pose.position.y #get y position from /odom topic

    info.vel_x=od.twist.twist.linear.x #get x linear velocity from /odom topic

    info.vel_z=od.twist.twist.angular.z #get z angular velocity from /odom topic

    pub.publish(info) # publish info on /posvel topic
    





      
    #while not rospy.is_shutdown():

        #client.wait_for_server() #synchronizing client and server

        #menu(client) #calling the menu function
        




		
#main()


rospy.init_node('planning_client2') #necessary to start the node

client=actionlib.SimpleActionClient('reaching_goal',PlanningAction) #necessary to start the client

rospy.Subscriber('/odom',Odometry,publish_function) #subscribing to /odom topic


        
b1.on_click(position) 
b2.on_click(cancel)
b3.on_click(reached_cancelled)

GridBox(children=[b1,b2,b3,b4],layout=Layout(width="80%",grid_template_rows="auto auto",
                  grid_template_columns="45% 45%", 
                  grid_template_areas='''"b1 b2"
                                     "b3 b4"'''))
